In [1]:
from time import time
import numpy as np
from keras.models import Model
from keras.optimizers import SGD

from sklearn.cluster import KMeans
from sklearn import metrics

from DEC import cluster_acc, ClusteringLayer, autoencoder
from IDEC import IDEC

Using TensorFlow backend.


In [2]:
#Arguments
dataset = 'mnist' #choices=['mnist', 'usps', 'reutersidf10k', 'pendigits']
n_clusters = 10
batch_size = 256
maxiter = 2e4
pretrain_epochs = 1 #200
gamma = 0.1
update_interval = 0
tol = 0.001
ae_weights = None#'./ae_weights'
save_dir = 'results/idec'

In [6]:
# load dataset

def load_dexNet():
    # the data, shuffled and split between train and test sets
    from keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x = np.concatenate((x_train, x_test))
    y = np.concatenate((y_train, y_test))
    x = x.reshape((x.shape[0], -1))
    x = np.divide(x, 255.)  # normalize as it does in DEC paper
    print('MNIST samples', x.shape)
    return x, y



optimizer = 'adam'  # SGD(lr=0.01, momentum=0.99)
from datasets import load_mnist, load_reuters, load_usps, load_pendigits
if dataset == 'mnist':  # recommends: n_clusters=10, update_interval=140
    x, y = load_mnist()
elif dataset == 'usps':  # recommends: n_clusters=10, update_interval=30
    x, y = load_usps('data/usps')
elif dataset == 'pendigits':
    x, y = load_pendigits('data/pendigits')
elif dataset == 'reutersidf10k':  # recommends: n_clusters=4, update_interval=20
    x, y = load_reuters('data/reuters')

if update_interval == 0:  # one epoch
    update_interval = int(x.shape[0]/batch_size)

('MNIST samples', (70000, 784))
1.0


In [ ]:
# Define IDEC model
idec = IDEC(dims=[x.shape[-1], 500, 500, 2000, 10], n_clusters=n_clusters)
idec.model.summary()

In [ ]:
t0 = time()

# Pretrain autoencoders before clustering
if ae_weights is None:
    idec.pretrain(x, batch_size=batch_size, epochs=pretrain_epochs, optimizer=optimizer)

# begin clustering, time not include pretraining part.

idec.compile(loss=['kld', 'mse'], loss_weights=[gamma, 1], optimizer=optimizer)
idec.fit(x, y=y, batch_size=batch_size, tol=tol, maxiter=maxiter,
         update_interval=update_interval, ae_weights=ae_weights, save_dir=save_dir)

# Show the final results
y_pred = idec.y_pred
print('acc:', cluster_acc(y, y_pred))
print('clustering time: %d seconds.' % int(time() - t0))

In [ ]:
# Show Cluster and Encoding output
    #Cluster
q, _ = idec.model.predict(x, verbose=0)
np.save('./clustering_layer.npy', q)
    #Encoding 
e = idec.extract_feature(x)
np.save('./encoding_layer.npy', e)